In [1]:
import mlflow
import mlflow.pytorch
import torch
import torch.nn as nn

# 1. Set the tracking URI 
mlflow.set_tracking_uri("http://mlflow:5000")

# 2. Create a new experiment with the correct shared volume artifact location
experiment_name = "pytorch-shared-volume"
try:
    # Try to create experiment with shared volume artifact location
    experiment_id = mlflow.create_experiment(
        name=experiment_name,
        artifact_location="file:///shared/mlflow-artifacts"
    )
    print(f"✅ Created new experiment: {experiment_name} (ID: {experiment_id})")
except mlflow.exceptions.MlflowException as e:
    if "already exists" in str(e):
        # Experiment already exists, get its ID
        experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
        print(f"✅ Using existing experiment: {experiment_name} (ID: {experiment_id})")
    else:
        raise e

# 3. Set the experiment
mlflow.set_experiment(experiment_name)

# 4. Create a simple PyTorch model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear = nn.Linear(10, 1)
    
    def forward(self, x):
        return self.linear(x)

# 5. Start MLflow run with the correct experiment
with mlflow.start_run(run_name="Fixed MLflow Integration Test"):
    # Log hyperparameters
    learning_rate = 0.01
    epochs = 10
    
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("epochs", epochs)
    
    # Create and train model
    model = SimpleModel()
    
    # Log metrics
    mlflow.log_metric("accuracy", 0.95)
    mlflow.log_metric("loss", 0.05)
    
    # This WILL work now! 🎉
    print("🔧 Logging model to shared volume...")
    mlflow.pytorch.log_model(model, "model")
    
    print("✅ Experiment logged to MLflow successfully!")
    print("🔗 View at: http://localhost:5000")
    print(f"📁 Artifacts stored in: /shared/mlflow-artifacts")

/opt/conda/lib/python3.10/site-packages/mlflow/utils/requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251
/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


✅ Using existing experiment: pytorch-shared-volume (ID: 6)
🔧 Logging model to shared volume...
✅ Experiment logged to MLflow successfully!
🔗 View at: http://localhost:5000
📁 Artifacts stored in: /shared/mlflow-artifacts
